In [ ]:
from pathlib import Path
import json
from pprint import pprint

from mihcsme_py import parse_excel_to_model, upload_metadata_to_omero, download_metadata_from_omero
from mihcsme_py.models import (
    MIHCSMEMetadata,
    AssayCondition,
    InvestigationInformation,
    StudyInformation,
    AssayInformation,
)

# Optional: For nice table display
import pandas as pd

print("✅ Imports successful!")

In [ ]:
# OMERO connection parameters (update these!)
OMERO_HOST = "localhost"  # Change this
OMERO_USER = "root"  # Change this
OMERO_PASSWORD = "omero"  # Change this (or use getpass)

# Target for upload
TARGET_TYPE = "Screen"  # or "Plate"
TARGET_ID = 13  # Change this to your Screen/Plate ID

print("⚠️  OMERO Upload Configuration:")
print(f"   Host: {OMERO_HOST}")
print(f"   User: {OMERO_USER}")
print(f"   Target: {TARGET_TYPE} ID {TARGET_ID}")
print("\n   ⚠️  Update these values before running!")

In [ ]:
# Uncomment to run the upload
import ezomero
# Connect to OMERO
print("🔌 Connecting to OMERO...")
conn = ezomero.connect(
    host=OMERO_HOST,
    user=OMERO_USER,
    password=OMERO_PASSWORD,
    secure=True,
    port=4064,
)
print("✅ Connected!")


In [ ]:
# 1. Parse Excel → Pydantic
original = parse_excel_to_model("test_MIHCSME.xlsx")
original


In [ ]:
# 2. Upload Pydantic → OMERO
upload_metadata_to_omero(conn, original, "Screen", 13,replace=True)



In [ ]:
# 3. Download OMERO → Pydantic
downloaded = download_metadata_from_omero(conn, "Screen", 51)


In [ ]:
# 4. Work with DataFrame
df =downloaded.to_dataframe()
pprint(df)
df['Gene Symbol lower'] = df['Gene Symbol'].str.lower() 
downloaded = downloaded.update_conditions_from_dataframe(df)
#upload to omero again
upload_metadata_to_omero(conn, downloaded, "Screen", 51,replace=True)



In [ ]:
# 5. Compare
assert downloaded.investigation_information.investigation_info.project_id == \
       original.investigation_information.investigation_info.project_id